# Energy Optimization API - Ejemplos Optimizados

Versión optimizada con llamadas concurrentes para reducir tiempo de ejecución.

In [1]:
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

import pandas as pd
import requests

BASE_URL = "http://localhost:8000"

## Predicción para diferentes horarios

In [2]:
def make_prediction(hour):
    """Hacer una predicción para una hora específica"""
    nsm = hour * 3600
    request_data = {
        "lagging_reactive_power": 23.45,
        "leading_reactive_power": 12.30,
        "co2": 0.05,
        "lagging_power_factor": 0.85,
        "leading_power_factor": 0.92,
        "nsm": nsm,
        "day_of_week": 1,
        "load_type": "Medium"
    }
    response = requests.post(f"{BASE_URL}/predict", json=request_data)
    if response.status_code == 200:
        result = response.json()
        return {
            "hora": f"{hour:02d}:00",
            "consumo_kwh": result['predicted_usage_kwh']
        }
    return None

# Ejecutar en paralelo
hours = [0, 6, 12, 18]
start_time = time.time()

with ThreadPoolExecutor(max_workers=4) as executor:
    futures = [executor.submit(make_prediction, hour) for hour in hours]
    predictions = [future.result() for future in as_completed(futures) if future.result()]

elapsed = time.time() - start_time

df_predictions = pd.DataFrame(sorted(predictions, key=lambda x: x['hora']))
print(f"\nConsumo por horario (tiempo: {elapsed:.3f}s):")
print(df_predictions.to_string(index=False))


Consumo por horario (tiempo: 21.132s):
 hora  consumo_kwh
00:00    41.141857
06:00    45.045675
12:00    44.456209
18:00    44.456209


## Predicción para diferentes tipos de carga

In [3]:
def make_prediction_by_load(load_type):
    """Hacer una predicción para un tipo de carga específico"""
    request_data = {
        "lagging_reactive_power": 23.45,
        "leading_reactive_power": 12.30,
        "co2": 0.05,
        "lagging_power_factor": 0.85,
        "leading_power_factor": 0.92,
        "nsm": 36000,
        "day_of_week": 1,
        "load_type": load_type
    }
    response = requests.post(f"{BASE_URL}/predict", json=request_data)
    if response.status_code == 200:
        result = response.json()
        return {
            "tipo_carga": load_type,
            "consumo_kwh": result['predicted_usage_kwh']
        }
    return None

# Ejecutar en paralelo
load_types = ["Light", "Medium", "Maximum"]
start_time = time.time()

with ThreadPoolExecutor(max_workers=3) as executor:
    futures = [executor.submit(make_prediction_by_load, lt) for lt in load_types]
    load_predictions = [future.result() for future in as_completed(futures) if future.result()]

elapsed = time.time() - start_time

df_load = pd.DataFrame(sorted(load_predictions, key=lambda x: load_types.index(x['tipo_carga'])))
print(f"\nConsumo por tipo de carga (tiempo: {elapsed:.3f}s):")
print(df_load.to_string(index=False))


Consumo por tipo de carga (tiempo: 21.071s):
tipo_carga  consumo_kwh
     Light    44.456209
    Medium    44.456209
   Maximum    44.456209


## Alternativa: Usar endpoint batch

In [4]:
# En lugar de múltiples llamadas, usa el endpoint batch
hours = [0, 6, 12, 18]
batch_request = {
    "predictions": [
        {
            "lagging_reactive_power": 23.45,
            "leading_reactive_power": 12.30,
            "co2": 0.05,
            "lagging_power_factor": 0.85,
            "leading_power_factor": 0.92,
            "nsm": hour * 3600,
            "day_of_week": 1,
            "load_type": "Medium"
        }
        for hour in hours
    ]
}

start_time = time.time()
response = requests.post(f"{BASE_URL}/predict/batch", json=batch_request)
elapsed = time.time() - start_time

if response.status_code == 200:
    result = response.json()
    predictions = [
        {
            "hora": f"{hours[i]:02d}:00",
            "consumo_kwh": pred['predicted_usage_kwh']
        }
        for i, pred in enumerate(result['predictions'])
    ]
    df = pd.DataFrame(predictions)
    print(f"\nConsumo por horario usando batch (tiempo: {elapsed:.3f}s):")
    print(df.to_string(index=False))
    print(f"\nTiempo de procesamiento API: {result['summary']['processing_time_ms']:.2f} ms")


Consumo por horario usando batch (tiempo: 21.063s):
 hora  consumo_kwh
00:00    41.141857
06:00    45.045675
12:00    44.456209
18:00    44.456209

Tiempo de procesamiento API: 13.24 ms
